# ChatPromptTemplate
- 대화 목록을 프롬프트로 주고자 할때 활용. ai와 대화를 할 수 있다.
- 메세지는 튜플(tuple)형식이며, (role, message)로 구성하여 리스토로 생성할 수 있다.

(role, message)구조

### role
- `"system"`: 시스템 설정 메세지. 주로 전역설정과 관련된 프롬프트. ai role이나, 페르소나등 임무등을 쥐어준다. 적용되는 범위가 대화 전체다.
- `"human"`: 사용자 입력 메세지
- `"ai"`: ai의 답변 메세지

### Message
- 우리의 지시사항, 전역설정 prompt, ai의 답변등 모두 될 수 있다.

In [36]:
from dotenv import load_dotenv
load_dotenv()

True

In [37]:
from langchain_teddynote import logging
logging.langsmith("CH02-Prompt")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH02-Prompt


In [38]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{country}의 수도는 어디인가요?")
chat_prompt

ChatPromptTemplate(input_variables=['country'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?'))])

In [39]:
chat_prompt.format(country="대한민국")

'Human: 대한민국의 수도는 어디인가요?'

In [40]:
chat_template = ChatPromptTemplate.from_messages(
    [
        #role, message
        ("system", "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name}입니다."),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}")
    ]
)
chat_template

ChatPromptTemplate(input_variables=['name', 'user_input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], template='당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name}입니다.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='반가워요!')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='안녕하세요! 무엇을 도와드릴까요?')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], template='{user_input}'))])

In [41]:
messages = chat_template.format_messages(
    name="방이", user_input="당신의 이름은 무엇입니까?"
)
messages


[SystemMessage(content='당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 방이입니다.'),
 HumanMessage(content='반가워요!'),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요?'),
 HumanMessage(content='당신의 이름은 무엇입니까?')]

In [42]:
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="llama3:8b")

In [43]:
llm.invoke(messages).content

"나의 이름은 방이(Bangi)입니다! 😊 Nice to meet you! I'm here to help and assist with any questions or topics you'd like to discuss. What's on your mind? 🤔"

In [44]:
chain = chat_template | llm

In [45]:
chain.invoke({"name": "방이", "user_input": "당신의 이름은 무엇입니까?"}).content

'저는 방이(Bangi)입니다! 친절한 AI 어시스턴트로, 다양한 질문과 요구에 응대하여 도움을 드리겠습니다. 반가워요! 😊'

# MessagePlaceholder
- 확정되지 않은 메세지나 대화를 나중에 넣어주기 위해 위치만 잡아주는 것이다.

In [46]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="conversation"),
        ('human', '지금까지의 대화를 {word_count} 단어로 요약합니다.')
    ]
)
chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[MessagesPlaceholder(variable_name='conversation'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['word_count'], template='지금까지의 대화를 {word_count} 단어로 요약합니다.'))])

##### `conversation` 대화목록을 나중에 추가하고자 할 때, `MessagesPlaceholder`를 사용할 수 있다.

In [48]:
# () tuple형태, {} dictionary형태, [] list형태

In [49]:
chat_prompt.format(
    word_count=5,
    conversation=[
        ("human", "안녕하세요! 저는 오늘 새로 입사한 뎨방입니다. 만나서 반갑습니다."),
        ('ai', "반가워요! 앞으로 잘 부탁드립니다!")
    ]
)

'Human: 안녕하세요! 저는 오늘 새로 입사한 뎨방입니다. 만나서 반갑습니다.\nAI: 반가워요! 앞으로 잘 부탁드립니다!\nHuman: 지금까지의 대화를 5 단어로 요약합니다.'

In [50]:
chain = chat_prompt | llm

In [51]:
chain.invoke(
    {
        "word_count":5,
        "conversation": [
            (
                "human",
                "안녕하세요! 저는 오늘 새로 입사한 뎨방입니다. 만나서 반갑습니다."
            ),
            ("ai", "반가워요! 앞으로 잘 부탁드립니다.")
        ]
    }
).content

'"안녕하세요 반가워요 등대화"'

In [52]:
conversation

NameError: name 'conversation' is not defined